In [1]:
import pandas as pd
import re
from joblib import Parallel, delayed

In [35]:
def clima(m,anio):
    
    '''Funcion que saca las medias de los datos de temperatura y precipitaciones en España
        La función además de descargar los ficheros quel tiempo configura el dataframe'''
    
    tiempo=pd.DataFrame(columns=['fecha','temp_max', 'temp_min', 'temp_media','precipitacion'] )
    
       
    lst=['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15'
         '16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31']
    
    for d in lst:
        try:
            print('carga')
            df=pd.read_excel(f'../tiempo/Aemet{anio}-{m}/Aemet{anio}-{m}-{d}.xls')
        except:
            print('No carga error')
            continue

        df.dropna(inplace=True)
                
        df.columns=df.loc[3]
                
        df.drop([3],axis=0, inplace=True)
                
        df['Temperatura máxima (ºC)']=df['Temperatura máxima (ºC)'].apply(lambda x: re.findall('\d+.\d+',x)[0])
        df['Temperatura mínima (ºC)']=df['Temperatura mínima (ºC)'].apply(lambda x: re.findall('\d+.\d+',x)[0])

        def to_int(val):
            """ Reconoce valores numericos y los transforma a enteros.
            """
            try:
                value = int(float(val))
            except ValueError:
                value = ""
            return value

        df['Temperatura máxima (ºC)']=df['Temperatura máxima (ºC)'].apply(to_int)
        df['Temperatura mínima (ºC)']=df['Temperatura mínima (ºC)'].apply(to_int)
                
        fecha='2017-01-01'
        temp_max=round(df['Temperatura máxima (ºC)'].mean(),2)
        temp_min=round(df['Temperatura mínima (ºC)'].mean(),2)
        temp_media=round(df['Temperatura media (ºC)'].mean(),2)
        precipitacion=round(df['Precipitación 00-24h (mm)'].mean(),2)

        tiempo = tiempo.append({'fecha': f'{anio}-{m}-{d}', 'temp_max':temp_max, 'temp_min':temp_min,
                        'temp_media':temp_media,'precipitacion':precipitacion}, ignore_index=True)
    
        
    return tiempo

    

In [42]:
anio=['2017','2018','2019','2020','2021','2022']
mes=['01','02','03','04','05','06','07','08','09','10','11','12']

In [43]:
final=[]
for a in anio:
    lst=Parallel(n_jobs=-1, verbose=True)(delayed(clima)(e,a) for e in mes)
    final.append(pd.concat(lst))
    
    

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    4.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    5.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    4.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    4.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    4.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    4.5s finished


In [45]:
tabla_tiempo=pd.concat(final)
tabla_tiempo

,fecha,temp_max,temp_min,temp_media,precipitacion
0,2017-01-01,10.11,4.36,5.48,0.03
1,2017-01-02,11.05,4.36,6.81,0.19
2,2017-01-03,13.13,4.55,8.61,0.46
3,2017-01-04,13.68,5.14,9.3,0.06
4,2017-01-05,12.9,4.36,8.4,0.07
...,...,...,...,...,...
23,2022-11-26,16.46,4.99,10.93,0.02
24,2022-11-27,16.33,4.93,10.78,1.84
25,2022-11-28,13.9,6.07,10.36,2.82
26,2022-11-29,13.8,5.28,9.82,1.1


In [46]:
tabla_tiempo.to_csv('../data/tabla_tiempo.csv', index=False,index_label=False)